In [3]:

from Models.MLPnetwork import MLPNetwork
from Models.Value import Value
from Trainer import Trainer
data = [
    ([2.0, 3.0], 0),
    ([1.0, 1.5], 0),
    ([2.5, 2.2], 0),
    ([3.0, 0.5], 1),
    ([4.0, 1.0], 1),
    ([3.5, 0.2], 1),
]

# Binary Cross-Entropy loss for sigmoid outputs
def bce_loss(prob, y_true):
    """
    Binary Cross-Entropy Loss for sigmoid outputs.
    
    prob: Value    # model output after sigmoid (a probability)
    y_true: int    # 0 or 1
    """

    # y * log(p)
    term1 = prob.log() if y_true == 1 else Value(0.0)

    # (1 - y) * log(1 - p)
    term2 = (1 - prob).log() if y_true == 0 else Value(0.0)

    # BCE = - (term1 + term2)
    return -(term1 + term2)
    


# creating the model with 1 hidden layer and 1 output
model = MLPNetwork(
    input_dim=2,
    n_neurons=[4, 1],          # [hidden_size, output_size]
    label="toy",
    activation_type="tanh",
    classification="sigmoid"
)

# Training loop (simple SGD)
learning_rate = 0.1
epochs = 200

for epoch in range(epochs):
    total_loss = Value(0.0)
    correct = 0

    for (x_raw, y_true) in data:
        # Forward pass
        probs = model.forward(x_raw)
        loss = bce_loss(probs[0], y_true)

        # Accumulate loss for reporting
        total_loss = total_loss + loss

        # Accuracy
        pred = 1 if probs[0].data > 0.5 else 0
        if pred == y_true:
            correct += 1

        # Backward + update (SGD step)
        model.zero_grad()     # reset grads BEFORE backward
        loss.backward()
        for p in model.parameters():
            p.data -= learning_rate * p.grad

    # Report average loss + accuracy
    if epoch % 20 == 0:
        avg_loss = total_loss.data / len(data)
        acc = correct / len(data)
        print(f"epoch {epoch:3d} | loss={avg_loss:.4f} | acc={acc:.2f}")

# test
print("\nFinal predictions:")
for (x_raw, y_true) in data:
    probs = model.forward(x_raw)
    pred = 1 if probs[0].data > 0.5 else 0
    print(f"x={x_raw}, y_true={y_true}, prob={probs[0].data:.3f}, pred={pred}")

AttributeError: 'MLPNetwork' object has no attribute 'forward'

In [ ]:
from Models.MLP import MLPNetwork
from Trainer.trainer import Trainer
data = [
    ([2.0, 3.0], [0,1]),
    ([1.0, 1.5], [0,1]),
    ([2.5, 2.2], [0,1]),
    ([3.0, 0.5], [0,1]),
    ([4.0, 1.0], [0,1]),
    ([3.5, 0.2], [0,1]),
]
predictors = []
labels = []

for x,y in data:
    predictors.append(x)
    labels.append(y)

model = MLPNetwork(input_dim=2, n_neurons=[4, 2], label="toy", activation_type="tanh", classification="none")
trainer = Trainer(model=model, epochs=100)
trainer.fit(predictors, labels)
trainer.test(predictors, labels)

Trainer initialized for regression task using LinearLoss loss.
Epoch 1/100, Loss: 0.6469
Epoch 20/100, Loss: 0.0320
Epoch 40/100, Loss: 0.0149
Epoch 60/100, Loss: 0.0107
Epoch 80/100, Loss: 0.0081
Epoch 100/100, Loss: 0.0063


0.005898927136473401

In [1]:
import random

def make_toy_data(n_per_class=100):
    X_real = []
    y_labels = []

    for _ in range(n_per_class):
        # class 0: around (-1, -1)
        x1 = -1 + random.gauss(0, 0.2)
        x2 = -1 + random.gauss(0, 0.2)
        X_real.append([x1, x2])
        y_labels.append(0)

    for _ in range(n_per_class):
        # class 1: around (1, 1)
        x1 = 1 + random.gauss(0, 0.2)
        x2 = 1 + random.gauss(0, 0.2)
        X_real.append([x1, x2])
        y_labels.append(1)

    return X_real, y_labels

In [ ]:
from Models.MLPGenerator import MLPGenerator
from Trainer.trainer import Trainer
import random

def make_toy_data(n_per_class=100):
    X_real = []
    y_labels = []

    for _ in range(n_per_class):
        # class 0: around (-1, -1)
        x1 = -1 + random.gauss(0, 0.2)
        x2 = -1 + random.gauss(0, 0.2)
        X_real.append([x1, x2])
        y_labels.append(0)

    for _ in range(n_per_class):
        # class 1: around (1, 1)
        x1 = 1 + random.gauss(0, 0.2)
        x2 = 1 + random.gauss(0, 0.2)
        X_real.append([x1, x2])
        y_labels.append(1)

    return X_real, y_labels

def to_one_hot(label, num_classes):
    return [1 if i == label else 0 for i in range(num_classes)]

def build_generator_training_data(X_real, y_labels, latent_dim, num_classes):
    X_train = []   # inputs to generator
    y_train = []   # targets (real x)

    for x, y in zip(X_real, y_labels):
        # sample random noise z
        z = [random.uniform(-1, 1) for _ in range(latent_dim)]
        # one-hot encode label
        y_vec = to_one_hot(y, num_classes)
        # concat [z, y_onehot] as input
        inp = z + y_vec

        X_train.append(inp)
        y_train.append(x)

    return X_train, y_train
# hyperparameters
latent_dim  = 4          # size of noise vector z
num_classes = 2          # y in {0, 1}
cond_dim    = num_classes
output_dim  = 2          # x is 2D
hidden_sizes = [16, 16]  # small MLP

# create generator model
gen_model = MLPGenerator(
    latent_dim=latent_dim,
    cond_dim=cond_dim,
    output_dim=output_dim,
    hidden_sizes=hidden_sizes,
    activation_type="tanh"
)

# create trainer (regression, so classification="none" in model)
trainer = Trainer(
    model=gen_model,
    optimizer=None,      # uses default SGD in your __init__
    loss_fn=None,        # uses default LinearLoss in your __init__
    epochs=50
)

Trainer initialized for regression task using LinearLoss loss.


In [3]:
# 1. real data
X_real, y_labels = make_toy_data(n_per_class=200)

# 2. build (input, target) pairs for generator
X_train, y_train = build_generator_training_data(
    X_real, y_labels,
    latent_dim=latent_dim,
    num_classes=num_classes
)

# 3. train
trainer.fit(X_train, y_train)

Epoch 1/50, Loss: 0.2782
Epoch 20/50, Loss: 0.0442
Epoch 40/50, Loss: 0.0426


In [5]:
def sample_z(latent_dim):
    return [random.uniform(-1, 1) for _ in range(latent_dim)]

# generate 5 new samples for class 0
print("Synthetic samples for class 0:")
for _ in range(5):
    z = sample_z(latent_dim)
    y_vec = to_one_hot(0, num_classes)      # condition on class 0
    x_hat = gen_model.generate(z, y_vec)
    print(x_hat)

# generate 5 new samples for class 1
print("\nSynthetic samples for class 1:")
for _ in range(5):
    z = sample_z(latent_dim)
    y_vec = to_one_hot(1, num_classes)      # condition on class 1
    x_hat = gen_model.generate(z, y_vec)
    print(x_hat)

Synthetic samples for class 0:
[Value: -1.0559758216501918, Grad: 0, Value: -0.9542930327942515, Grad: 0]
[Value: -0.9809952181716437, Grad: 0, Value: -1.0358120493219025, Grad: 0]
[Value: -1.0725440286621213, Grad: 0, Value: -0.9512467152491785, Grad: 0]
[Value: -1.0064335722720055, Grad: 0, Value: -1.0117034090288402, Grad: 0]
[Value: -0.9000556539326645, Grad: 0, Value: -1.049317561478556, Grad: 0]

Synthetic samples for class 1:
[Value: 0.9608756327356689, Grad: 0, Value: 0.9623279563241465, Grad: 0]
[Value: 1.0562431388054598, Grad: 0, Value: 0.9640717531464561, Grad: 0]
[Value: 1.014131203869769, Grad: 0, Value: 0.9649806802962615, Grad: 0]
[Value: 1.1632670713306237, Grad: 0, Value: 0.8835455789731771, Grad: 0]
[Value: 1.0035622775327868, Grad: 0, Value: 0.9617033974953391, Grad: 0]


In [1]:
import pandas as pd
from Utils import data_builder
 
df = pd.read_csv("/Users/peter/Library/CloudStorage/OneDrive-Personal/Documents/MDS_UBC/iris.csv")

df["Species"] = df["Species"].astype("category").cat.codes

X_train, y_train = data_builder.build_generator_training_data(
    df.iloc[:, :-1].values.tolist(),
    df["Species"].tolist(),
    latent_dim=4
)


from Models.MLPGenerator import MLPGenerator
from Trainer.trainer import Trainer

model = MLPGenerator(
    latent_dim=4,
    cond_dim=1,
    output_dim=4,
    hidden_sizes=[16, 16],
    activation_type="tanh"
)

train = Trainer(model=model, epochs=20)
train.fit(X_train, y_train)


Trainer initialized for regression task using LinearLoss loss.
Epoch 1/20, Loss: 1.9042
Epoch 20/20, Loss: 0.1629


In [2]:
import random
seed_vectors = [[random.uniform(-1, 1) for _ in range(4)] for _ in range(len(df["Species"]))]
random_labels = df["Species"].tolist()

list_of_generated = []
for z, y in zip(seed_vectors, random_labels):
    list_of_generated.append((model.generate(z, [y]), y))
    

In [8]:
list_of_generated

[([5.275207124823355,
   3.452802953243714,
   1.630664621221457,
   0.13585715845030505],
  0),
 ([5.083191841224717,
   3.295819067174452,
   1.7386378178509143,
   0.10451614458751872],
  0),
 ([5.1542399749569086,
   3.6500859837512625,
   1.5229015302901336,
   0.11106235812560017],
  0),
 ([5.071925157871115,
   3.2523337531786343,
   1.8186026675909797,
   0.16269299332196469],
  0),
 ([5.251641028885373,
   3.2757570924101276,
   1.5756057513845516,
   0.2598734309839952],
  0),
 ([4.416155139129472,
   3.3989735226549933,
   1.3726169104132195,
   0.34047216980861345],
  0),
 ([4.772154332688933,
   3.4773903779374105,
   1.7077089357432913,
   0.6075754307003657],
  0),
 ([4.778021455984011,
   3.4489801887522398,
   1.5159990262258176,
   -0.0851797571335774],
  0),
 ([5.195741178790682,
   3.672338665914862,
   1.2854420838708858,
   0.019967059253662878],
  0),
 ([5.093050710590327,
   3.5558448881911877,
   1.4104926898234642,
   0.02842209079277519],
  0),
 ([4.869039936

In [6]:
import pandas as pd

feature_cols = df.columns.tolist()

generated_df = pd.DataFrame(
    [x + [label] for x, label in list_of_generated],
    columns=feature_cols
)

print(generated_df.head())

   Sepal.Length  Sepal.Width  Petal.Length  Petal.Width  Species
0      5.275207     3.452803      1.630665     0.135857        0
1      5.083192     3.295819      1.738638     0.104516        0
2      5.154240     3.650086      1.522902     0.111062        0
3      5.071925     3.252334      1.818603     0.162693        0
4      5.251641     3.275757      1.575606     0.259873        0


In [11]:
generated_df.head()

print(generated_df["Petal.Length"].mean())
print(df["Petal.Length"].mean())
print(generated_df["Petal.Width"].mean())
print(df["Petal.Width"].mean())



3.902901325996727
3.7580000000000005
1.2184834269048592
1.1993333333333336


In [ ]:
from Models.MLPGenerator import MLPNetwork
model_prediction = MLPNetwork(4, [16,16,3], "iris_pred", "tanh", classification="softmax")
trainer = Trainer(model=model_prediction, epochs=50, loss_fn="cross_entropy")
predictors = generated_df.iloc[:, :-1].values.tolist()
labels = generated_df["Species"].astype(int).tolist()
loss_fn = 
trainer.fit(predictors, labels)

Trainer initialized for classification task using str loss.


TypeError: 'str' object is not callable